In [1]:
import json
import os
from PIL import Image

def convert_labelme_to_yolo(json_path):
    # 대응되는 이미지 파일 찾기
    img_path_jpg = json_path.replace('.json', '.jpg')
    img_path_png = json_path.replace('.json', '.png')

    if os.path.exists(img_path_jpg):
        img_path = img_path_jpg
    elif os.path.exists(img_path_png):
        img_path = img_path_png
    else:
        print(f"❌ 이미지 파일 없음: {json_path}")
        return

    # 이미지 크기 불러오기
    image = Image.open(img_path)
    w, h = image.size

    # JSON 로딩
    with open(json_path, 'r') as f:
        data = json.load(f)

    txt_lines = []
    for shape in data['shapes']:
        label = 0  # roscar 단일 클래스
        points = shape['points']
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)

        x_center = ((x_min + x_max) / 2) / w
        y_center = ((y_min + y_max) / 2) / h
        width = (x_max - x_min) / w
        height = (y_max - y_min) / h

        txt_lines.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # .txt 파일로 저장
    txt_path = json_path.replace('.json', '.txt')
    with open(txt_path, 'w') as f:
        f.write('\n'.join(txt_lines))

    print(f"✅ 변환 완료: {os.path.basename(txt_path)}")


# 전체 디렉토리 순회
base_dir = "/home/usou/Downloads/saved_frames"
for subdir, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.json'):
            json_full_path = os.path.join(subdir, file)
            convert_labelme_to_yolo(json_full_path)


✅ 변환 완료: 100_150_00108.txt
✅ 변환 완료: 100_150_00164.txt
✅ 변환 완료: 100_150_00131.txt
✅ 변환 완료: 100_150_00199.txt
✅ 변환 완료: 100_150_00055.txt
✅ 변환 완료: 100_150_00187.txt
✅ 변환 완료: 100_150_00074.txt
✅ 변환 완료: 100_150_00007.txt
✅ 변환 완료: 100_150_00172.txt
✅ 변환 완료: 100_150_00029.txt
✅ 변환 완료: 100_150_00185.txt
✅ 변환 완료: 100_150_00008.txt
✅ 변환 완료: 100_150_00097.txt
✅ 변환 완료: 100_150_00140.txt
✅ 변환 완료: 100_150_00139.txt
✅ 변환 완료: 100_150_00175.txt
✅ 변환 완료: 100_150_00083.txt
✅ 변환 완료: 100_150_00146.txt
✅ 변환 완료: 100_150_00069.txt
✅ 변환 완료: 100_150_00005.txt
✅ 변환 완료: 100_150_00196.txt
✅ 변환 완료: 100_150_00137.txt
✅ 변환 완료: 100_150_00059.txt
✅ 변환 완료: 100_150_00052.txt
✅ 변환 완료: 100_150_00013.txt
✅ 변환 완료: 100_150_00125.txt
✅ 변환 완료: 100_150_00024.txt
✅ 변환 완료: 100_150_00122.txt
✅ 변환 완료: 100_150_00195.txt
✅ 변환 완료: 100_150_00109.txt
✅ 변환 완료: 100_150_00009.txt
✅ 변환 완료: 100_150_00031.txt
✅ 변환 완료: 100_150_00178.txt
✅ 변환 완료: 100_150_00016.txt
✅ 변환 완료: 100_150_00079.txt
✅ 변환 완료: 100_150_00143.txt
✅ 변환 완료: 100_150_00177.txt
✅

# 학습 검증 데이터 나누기


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/home/usou/Downloads/saved_frames"
distances = ["0_50", "50_100", "100_150", "150_200"]

for dist in distances:
    img_dir = os.path.join(base_dir, dist)
    files = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    for mode, file_list in zip(["train", "val"], [train_files, val_files]):
        for f in file_list:
            txt = f.replace('.jpg', '.txt')

            # 이미지 복사
            dst_img_dir = f"/home/usou/roscar_dataset/{dist}/images/{mode}"
            os.makedirs(dst_img_dir, exist_ok=True)
            shutil.copy(os.path.join(img_dir, f), os.path.join(dst_img_dir, f))

            # 라벨 복사
            dst_lbl_dir = f"/home/usou/roscar_dataset/{dist}/labels/{mode}"
            os.makedirs(dst_lbl_dir, exist_ok=True)
            shutil.copy(os.path.join(img_dir, txt), os.path.join(dst_lbl_dir, txt))


✅ Python 코드로 거리별 data.yaml 자동 생성 (Jupyter용)

In [4]:
import os

base_dir = "/home/usou/roscar_dataset"
distances = ["0_50", "50_100", "100_150", "150_200"]

for dist in distances:
    folder_path = os.path.join(base_dir, dist)
    yaml_path = os.path.join(folder_path, "data.yaml")

    yaml_content = f"""path: {folder_path}
train: images/train
val: images/val

nc: 1
names: ['roscar']
"""

    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"✅ 생성 완료: {yaml_path}")


✅ 생성 완료: /home/usou/roscar_dataset/0_50/data.yaml
✅ 생성 완료: /home/usou/roscar_dataset/50_100/data.yaml
✅ 생성 완료: /home/usou/roscar_dataset/100_150/data.yaml
✅ 생성 완료: /home/usou/roscar_dataset/150_200/data.yaml


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/home/usou/Downloads/saved_frames"
output_dir = "/home/usou/roscar_dataset"
img_train_dir = os.path.join(output_dir, "images/train")
img_val_dir = os.path.join(output_dir, "images/val")
lbl_train_dir = os.path.join(output_dir, "labels/train")
lbl_val_dir = os.path.join(output_dir, "labels/val")

os.makedirs(img_train_dir, exist_ok=True)
os.makedirs(img_val_dir, exist_ok=True)
os.makedirs(lbl_train_dir, exist_ok=True)
os.makedirs(lbl_val_dir, exist_ok=True)

all_images = []
distances = ["0_50", "50_100", "100_150", "150_200"]
for dist in distances:
    folder = os.path.join(base_dir, dist)
    imgs = [f for f in os.listdir(folder) if f.endswith('.jpg')]
    all_images += [(os.path.join(folder, f), os.path.join(folder, f.replace('.jpg', '.txt'))) for f in imgs]

train, val = train_test_split(all_images, test_size=0.2, random_state=42)

for img_path, txt_path in train:
    shutil.copy(img_path, os.path.join(img_train_dir, os.path.basename(img_path)))
    shutil.copy(txt_path, os.path.join(lbl_train_dir, os.path.basename(txt_path)))

for img_path, txt_path in val:
    shutil.copy(img_path, os.path.join(img_val_dir, os.path.basename(img_path)))
    shutil.copy(txt_path, os.path.join(lbl_val_dir, os.path.basename(txt_path)))

# data.yaml 생성
with open(os.path.join(output_dir, "data.yaml"), 'w') as f:
    f.write(f"""path: {output_dir}
train: images/train
val: images/val

nc: 1
names: ['roscar']
""")

print("✅ 전체 거리 통합 학습 데이터셋 구성 완료")


✅ 전체 거리 통합 학습 데이터셋 구성 완료


# person_detection 에서 face 클래스 제거

In [3]:
import os

base_dir = "/home/usou/Downloads/person detection.v2i.yolov8"
label_dirs = [
    os.path.join(base_dir, "train", "labels"),
    os.path.join(base_dir, "valid", "labels")
]

for label_dir in label_dirs:
    for fname in os.listdir(label_dir):
        if fname.endswith('.txt'):
            fpath = os.path.join(label_dir, fname)
            with open(fpath, 'r') as f:
                lines = f.readlines()
            # class 1 (person)만 남기고, class index 0으로 리라벨링
            person_only = [l for l in lines if l.startswith('1 ')]
            with open(fpath, 'w') as f:
                f.writelines([l.replace('1 ', '0 ', 1) for l in person_only])


In [5]:
import os

base = "/home/usou/Downloads/person detection.v2i.yolov8"
splits = ["train", "valid"]

for split in splits:
    label_dir = os.path.join(base, split, "labels")
    for file in os.listdir(label_dir):
        if not file.endswith(".txt"):
            continue

        file_path = os.path.join(label_dir, file)
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:
                parts[0] = '0'  # 클래스 번호를 '0'으로 고정
                new_lines.append(' '.join(parts) + '\n')

        if new_lines:
            with open(file_path, "w") as f:
                f.writelines(new_lines)


In [10]:
from ultralytics import YOLO
import cv2

# 학습된 모델 로드 (경로는 상황에 맞게 수정)
model = YOLO("runs/detect/train5/weights/best.pt")

# 웹캠 열기 (0은 기본 카메라)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ 웹캠을 열 수 없습니다.")
    exit()

print("🎥 웹캠 스트리밍 시작... ESC 누르면 종료")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ 프레임 수신 실패")
        continue

    # 모델 추론 (conf=0.5 이상만 표시)
    results = model(frame, conf=0.8, verbose=False)
    annotated_frame = results[0].plot()

    cv2.imshow("YOLOv8 Webcam Detection", annotated_frame)

    # ESC 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


🎥 웹캠 스트리밍 시작... ESC 누르면 종료


In [7]:
import os

label_dir = "/home/usou/Downloads/person detection.v2i.yolov8"
for sub in ["train", "valid"]:
    path = os.path.join(label_dir, sub, "labels")
    for file in os.listdir(path):
        if file.endswith(".txt"):
            fpath = os.path.join(path, file)
            with open(fpath, "r") as f:
                lines = f.readlines()
            with open(fpath, "w") as f:
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        parts[0] = "0"  # 모든 객체를 'person'
                        f.write(" ".join(parts) + "\n")


In [11]:
from ultralytics import YOLO
import cv2

# 학습된 모델 로드 (경로는 상황에 맞게 수정)
model = YOLO("runs/detect/train_person_only2/weights/best.pt")

# 웹캠 열기 (0은 기본 카메라)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ 웹캠을 열 수 없습니다.")
    exit()

print("🎥 웹캠 스트리밍 시작... ESC 누르면 종료")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ 프레임 수신 실패")
        continue

    # 모델 추론 (conf=0.5 이상만 표시)
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()

    cv2.imshow("YOLOv8 Webcam Detection", annotated_frame)

    # ESC 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


🎥 웹캠 스트리밍 시작... ESC 누르면 종료


In [9]:
import cv2
from ultralytics import YOLO

model = YOLO('runs_yolov8/crowd_yolov8n/weights/best.pt')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # BGR → RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLOv8 추론 (conf 낮춤)
    results = model.predict(rgb_frame, imgsz=640, conf=0.3, verbose=False)

    # 시각화 결과 얻기
    annotated_frame = results[0].plot()

    cv2.imshow("YOLO Person Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
from ultralytics import YOLO

# COCO 80클래스로 학습된 YOLOv8n 불러오기
model = YOLO("yolov8n.pt")

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV는 BGR이므로 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 예측 수행
    results = model.predict(frame, imgsz=640, conf=0.9, verbose=False)

    # 시각화된 결과 그리기
    annotated_frame = results[0].plot()

    # 출력
    cv2.imshow("YOLOv8 COCO Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


# coco 데이터셋 10%만 사용한 person dection 모델(epoch 5)

In [7]:
import cv2
from ultralytics import YOLO

# 학습된 모델 불러오기
model = YOLO("/home/usou/runs_yolov8/person_detection_yolov8n_b243/weights/best.pt")

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 예측 수행
    results = model.predict(frame, conf=0.5, imgsz=640)

    # 결과 시각화
    annotated_frame = results[0].plot()

    # 화면에 출력
    cv2.imshow("Person Detection", annotated_frame)

    # 종료 조건: 'q' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 정리
cap.release()
cv2.destroyAllWindows()



0: 480x640 2 persons, 10.1ms
Speed: 0.9ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 5.7ms
Speed: 2.5ms preprocess, 5.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 7.0ms
Speed: 1.4ms preprocess, 7.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 5.9ms
Speed: 1.5ms preprocess, 5.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 6.8ms
Speed: 1.0ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 4.9ms
Speed: 1.4ms preprocess, 4.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 6.6ms
Speed: 1.1ms preprocess, 6.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 6.1ms
Speed: 1.7ms preprocess, 6.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640